<a href="https://colab.research.google.com/github/smozley/austinAIallianceintensive/blob/main/1_LLM_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM API Integration

In this demo, we will send an LLM a prompt template and see what our response looks like

Let's use the same prompt template we just used as an example:
```
Given the following data for {{ticker}}, predict whether the stock will go up, down, or stay stable over the next {{prediction_window}} days. Justify your answer.

Earnings: {{earnings_summary}}
News: {{news_summary}}
History: {{price_trend}}

Respond with: ["Up", "Down", or "Stable"] and a brief explanation.
```

Variables needed:
- `ticker`
- `prediction_window`
- `earnings_summary`
- `news_summary`
- `price_trend`

In [ ]:
# Let's manually define the first two.
ticker = "TSLA"
prediction_window = 10

## Initial Setup

### Claude Setup

In [ ]:
# Let's install our dependencies first.
!pip install anthropic

In [ ]:
# First let's establish our client. For this we're going to use Anthropic's APIs
# Be sure to set your ANTHROPI_API_KEY in the secrets tab.
import anthropic
from google.colab import userdata

client = anthropic.Anthropic(api_key=userdata.get('ANTHROPIC_API_KEY'))

# Let's make sure it's working by sending a quick message:
message = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude"}
    ]
)
print(message.content[0].text)

In [ ]:
# Now let's create a more abstract function that will allow us to send a model ID with our prompts.
from pydantic import BaseModel
from enum import StrEnum

class Model(StrEnum):
  SM = 'claude-3-5-haiku-20241022'
  MD = 'claude-sonnet-4-20250514'
  LG = 'claude-opus-4-20250514'

class Question(BaseModel):
  model: Model
  prompt: str


def claude(question: Question) -> str:
  message = client.messages.create(
    model=question.model.value,
    max_tokens=1024,
    messages=[
        {"role": "user", "content": question.prompt}
    ]
  )
  return message.content[0].text

In [ ]:
# Let's see what claude knows from the top of it's head
question = Question(model=Model.SM, prompt=f"What's the latest information you have about {ticker}")

response = claude(question)

print(response)

### Stock Market API Setup

Go to [AlphVantage's Website](https://www.alphavantage.co/) and create a free API key.

In [ ]:
AV_API_KEY = userdata.get('AV_API_KEY')

In [ ]:
import requests

# Let's test and make sure we get data
url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={ticker}&apikey={AV_API_KEY}'
r = requests.get(url)
data = r.json()

print(data)

## Prompt Templates

Here are the variables we still need to get before sending a templated prompt to the LLM

- `earnings_summary`
- `news_summary`
- `price_trend`

In [ ]:
# First let's create a function that will get us the recent earnings from AV and then generate
# a summary of the recent earnings

def get_earnings_summary(ticker: str) -> str:
  # Make the request to AV
  url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={ticker}&apikey={AV_API_KEY}'
  r = requests.get(url)
  data = r.json()

  # Now let's ask the LLM to create a short summary of our information
  question = Question(model=Model.SM, prompt=f"Here's the lastest information on {ticker}'s earnings. Today's date is 2025-08-04. Create a short summary of what you see here: {data}")
  response = claude(question)

  return response

# Let's test
earnings_summary = get_earnings_summary(ticker)
print(earnings_summary)

In [ ]:
# Next let's get the news summary in the same way we got the earnings summary

def get_news_summary(ticker: str) -> str:
  # Make the request to AV
  url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&apikey={AV_API_KEY}'
  r = requests.get(url)
  data = r.json()

  # Summarize
  question = Question(model=Model.SM, prompt=f"Here's the lastest information on {ticker}'s news including the sentiment. Create a short summary of what you see here: {data}")
  response = claude(question)

  return response

# Let's test
news_summary = get_news_summary(ticker)
print(news_summary)

In [ ]:
# Finally let's get the price trend

def get_price_trend(ticker: str, days: int) -> dict:
  # Make the request to AV
  url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey={AV_API_KEY}'
  r = requests.get(url)
  data = r.json()
  time_series = data.get("Time Series (Daily)", {})
  sorted_dates = sorted(time_series.keys(), reverse=True)  # most recent first

  # Truncate to the most recent `days` entries
  prices = {date: time_series[date] for date in sorted_dates[:days]}

  return prices

# Let's test
price_trend = get_price_trend(ticker, prediction_window)
print(price_trend)

Now that we have all the data we need let's get to templating

In [ ]:
prompt = f"""
Given the following data for {ticker}, predict whether the stock will go up, down, or stay stable over the next 2 days, based on the last {prediction_window} days of data. Justify your answer.

## Earnings:
{earnings_summary}

## News:
{news_summary}

## History:
{price_trend}

Respond with: **Up**, **Down**, or **Stable** and a brief explanation.
"""

question = Question(model=Model.MD, prompt=prompt)
response = claude(question)

print(response)

## Streaming vs. Non-Streaming

In [ ]:
from typing import Generator

# Let's create another abstract function to handle streams
def stream_claude(question: Question) -> Generator[str, None, None]:
  # Only MD and LG support streaming
  assert question.model in [Model.MD, Model.LG]

  with client.messages.stream(
        model=question.model.value,
        max_tokens=1024,
        messages=[{"role": "user", "content": question.prompt}]
    ) as stream:
        for text in stream.text_stream:
            yield text

In [ ]:
for chunk in stream_claude(question):
    print(chunk, end="", flush=True)

## JSON Output

In [ ]:
!pip install instructor

In [ ]:
# Let's create another abstract function to handle streams

# For this we will use an SDK called instructor that acts as a wrapper around AI clients and does things that they don't natively support like JSON mode.
import instructor
from typing import Type, TypeVar

instructor_client = instructor.from_anthropic(client=client)

T = TypeVar("T", bound=BaseModel)

def generate_structured_response(question: Question, response_model: Type[T]) -> T:
    return instructor_client.chat.completions.create(
        model=question.model.value,
        max_tokens=1024,
        messages=[{"role": "user", "content": question.prompt}],
        response_model=response_model,
    )

In [ ]:
# Now let's define a BaseModel and see how our response comes out.

# Our desired model
class StockInformation(BaseModel):
  ticker: str
  technical_analysis: str
  fundamental_analysis: str
  market_sentiment: str

# Our response
response = generate_structured_response(question, response_model=StockInformation)
print("Ticker:", response.ticker)
print(" ")
print("Technical Analysis:", response.technical_analysis)
print(" ")
print("Fundamental Analysis:", response.fundamental_analysis)
print(" ")
print("Market Sentiment:", response.market_sentiment)

In [ ]:
response.model_dump json

# Tool Use

## Tool & Schema Definition

In [ ]:
# First, let's define the tool input models, The LLM will use this and know what to input.
from pydantic import Field, BaseModel
from typing import Any, Callable


class GetNewsInput(BaseModel):
    ticker: str = Field(..., description="Stock ticker symbol, e.g., TSLA")

class GetEarningsInput(BaseModel):
    ticker: str = Field(..., description="Stock ticker symbol, e.g., TSLA")

class GetPriceTrendInput(BaseModel):
    ticker: str = Field(..., description="Stock ticker symbol, e.g., TSLA")
    days: int = Field(..., description="Number of past trading days to analyze")

# Now let's rewrite the existing functions that will do the same thing except this time, we will not use the LLM to summarize.
# These are the callable tool functions (think of these as handlers)
def get_news(input: GetNewsInput) -> str:
  url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={input.ticker}&apikey={AV_API_KEY}'
  r = requests.get(url)
  data = r.json()
  return str(data)

def get_earnings(input: GetEarningsInput) -> str:
  url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={input.ticker}&apikey={AV_API_KEY}'
  r = requests.get(url)
  data = r.json()
  return str(data)

def get_price_trends(input: GetPriceTrendInput) -> str:
  url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={input.ticker}&apikey={AV_API_KEY}'
  r = requests.get(url)
  data = r.json()
  time_series = data.get("Time Series (Daily)", {})
  sorted_dates = sorted(time_series.keys(), reverse=True)  # most recent first
  prices = {date: time_series[date] for date in sorted_dates[:input.days]}
  return str(prices)


# Let's create a function that will generate our schma.
def to_claude_tool(name: str, description: str, input_model: type[BaseModel]) -> dict:
    return {
        "name": name,
        "description": description,
        "input_schema": input_model.model_json_schema()
    }

# Define tools for Claude this will generate a list of the tool input schema that Claude can read from.
claude_tools = [
    to_claude_tool("get_news", "Get the latest news sentiment information for a given stock ticker", GetNewsInput),
    to_claude_tool("get_earnings", "Get the latest earnings for a given stock ticker", GetEarningsInput),
    to_claude_tool("get_price_trends", "Get the recent price trend for a stock over N days", GetPriceTrendInput),
]


TOOL_FUNCTIONS: dict[str, Callable[[Any], str]] = {
    "get_news": get_news,
    "get_earnings": get_earnings,
    "get_price_trends": get_price_trends,
}

## Streaming with Tools

In [ ]:
# Create a system prompt that will have a specific instruction to the LLM about generating the stock ticker.
system_prompt = "You are an assistant that aids with stock market research. If you decide to use a tool, you must first determine (from your own knowledge) the ticker of the company that you want to research"

In [ ]:
# Now let's create another abstract function that will create a streaming request but with tools.
def stream_claude_with_tools(question: Question) -> Generator[str, None, None]:
    """Stream Claude response with tool support"""
    assert question.model in [Model.MD, Model.LG]

    messages = [{"role": "user", "content": question.prompt}]

    while True:
        # First, make a non-streaming call to get the complete response
        response = client.messages.create(
            model=question.model.value,
            max_tokens=1024,
            tools=claude_tools,
            tool_choice={"type": "auto"},
            messages=messages,
            system=system_prompt
        )

        # Stream the text content to the user
        has_text = False
        for block in response.content:
            if block.type == "text":
                yield block.text
                has_text = True

        # Extract tool calls
        tool_calls = [block for block in response.content if block.type == "tool_use"]

        # Add assistant response to messages (this must include ALL content blocks)
        messages.append({
            "role": "assistant",
            "content": response.content
        })

        # If no tools were called, we're done
        if not tool_calls:
            break

        yield "\n\n[Executing tools...]\n\n"

        # Execute all tools and collect results
        tool_results = []

        for tool_call in tool_calls:
            tool_name = tool_call.name
            tool_input = tool_call.input
            tool_id = tool_call.id

            yield f"[Calling {tool_name} with {tool_input}, id: {tool_id}]\n"

            if tool_name in TOOL_FUNCTIONS:
                try:
                    # Create the appropriate input object
                    if tool_name == "get_news":
                        input_obj = GetNewsInput(**tool_input)
                    elif tool_name == "get_earnings":
                        input_obj = GetEarningsInput(**tool_input)
                    elif tool_name == "get_price_trends":
                        input_obj = GetPriceTrendInput(**tool_input)
                    else:
                        raise ValueError(f"Unknown tool: {tool_name}")

                    # Execute the tool function
                    result = TOOL_FUNCTIONS[tool_name](input_obj)
                    yield f"[Tool result received: {len(result)} characters]\n"

                    # Add to results with exact matching ID
                    tool_results.append({
                        "type": "tool_result",
                        "tool_use_id": tool_id,  # This must match exactly
                        "content": result
                    })

                except Exception as e:
                    error_msg = f"Tool execution error: {str(e)}"
                    yield f"[{error_msg}]\n"

                    # Add error result with exact matching ID
                    tool_results.append({
                        "type": "tool_result",
                        "tool_use_id": tool_id,  # This must match exactly
                        "content": error_msg,
                        "is_error": True
                    })
            else:
                # Tool not found - still need to provide a result
                error_msg = f"Tool '{tool_name}' not implemented"
                yield f"[{error_msg}]\n"
                tool_results.append({
                    "type": "tool_result",
                    "tool_use_id": tool_id,
                    "content": error_msg,
                    "is_error": True
                })

        # Ensure we have a tool result for every tool call
        if len(tool_results) != len(tool_calls):
            yield f"[ERROR: Mismatch - {len(tool_calls)} tool calls but {len(tool_results)} results]\n"
            break

        # Add tool results as user message
        messages.append({
            "role": "user",
            "content": tool_results
        })

        yield "\n[Analyzing results...]\n\n"

In [ ]:
prompt = "What's the last close price for Google?"

question = Question(model=Model.LG, prompt=prompt)

for chunk in stream_claude_with_tools(question):
    print(chunk, end="", flush=True)